How Python Assignment Works
===========================

In this lecture we take a deeper look at how assignment works in Python.  This might seem too technical, but having a good mental model of how things work is important for understanding what's going on when things don't quite work the way you might expect.

Assignment of a simple object
-----------------------------

Let's look at a simple example:

    x = 500
    y = x
    y = 'foo'
    print(x)

We can step through what's happening:

In [ ]:
x = 500

in executing this line, we created an integer object with value 500 somewhere in memory when the right hand side of the assignment is evaluated, and then the equals assignment creates, or binds, a reference to it in the current namespace.

We can see the namespace with IPython's `%whos` magic command:

In [ ]:
%whos

and every object in Python has a unique identifier which you can see with the `id()` builtin (in standard Python, this is the memory location of the object, but you shouldn't rely on that - the only guarantee is that at any given moment all ids are unique):

In [ ]:
id(x)

Now let's look at the next line:

In [ ]:
y = x

The righthand side is `x`, so that causes a look-up of the variable x in the local namespace, which returns the reference to the integer object.  This is then bound to the variable `y` on the left hand side.

We can see the state of the namespace:

In [ ]:
%whos

but also that the identity of the object referred to by `y` is the same as the one referred to by `x`:

In [ ]:
id(y)

The third line:

In [ ]:
y = 'foo'

creates a new string object on the right hand side with the value `'foo'` in it, and the assignment binds it to the variable `y` in the local namespace, removing the binding that `y` had to 500.  So our namespace looks like:

In [ ]:
%whos

but we also see that `y` now refers to a completely different obejct:

In [ ]:
id(y)

Finally the print statement

In [ ]:
print(x)

looks up the object referred to by x, and displays a text representation of it.

Assignment of a container object
--------------------------------

Let's compare this with the following similar, but slightly different code:

    x = [500, 501, 502]
    y = x
    y[1] = 600
    print(x)
    y = [700, 800]

We start with:

In [ ]:
x = [500, 501, 502]

Python evaluates the right-hand side first, creating objects for the numbers 500, 501 and 502 first, and then creating a list object which has the references to those objects bound to the indices 0, 1 and 2 respectively.  Finally, a reference to the new list object is bound to `x`.

Checking the ids, you see we have:

In [ ]:
id(x)

In [ ]:
id(x[0])

In [ ]:
id(x[1])

In [ ]:
id(x[2])

So there are indeed 4 different objects here.

Moving on to the next line, this works much like the line in the simple example:

In [ ]:
y = x

the variable reference on the right is evaluated and the object bound to the variable on the left.

So our namespace looks like this:

In [ ]:
%whos

but just like the simple example, `x` and `y` refer to the same object:

In [ ]:
id(y) == id(x)

So the next line is a little different.  In previous assignments we have had a simple variable on the left, and assignment has meant "bind this variable in the namespace to the object on the right".  But here we have `y` with an index on the left.

This can be informally thought of as being the equivalent of a function call something like:

    set_item(y, 1, 600)

This is saying that I need to set an item in `y`, at index `1` and with value `600`.

As an aside for the curious advanced student, the actual equivalent statement would be `list.__setitem__(y, 1, 600)` (if you understand why this is equivalent, then you pretty much understand Python's object model), but the idea is encapsulated well by a simple function call.

So when we execute:

In [ ]:
y[1] = 600

Python creates a new integer object with value 600 for the value on the right, a new integer object with value 1 for the index (actually, 1 already exists, but we can pretend we created it), and looks up the object referred to by `y`, which is the Python list.

Then the `set_item` function changes the list so that index 1 is now bound to 600, and the binding of index 1 to 501 is dropped.

Now since 501 and 1 have nothing referring to them, they are no longer needed, and Python can actually garbage collect the objects, freeing the memory.  It turns out that 1 would never be garbage collected, because of internal implementation details of the Python runtime, but 501 would be.

So we have changed the contents of the list at index 1.  And when it comes time to print out x:

In [ ]:
print(x)

we look up the object bound to `x`, which is our list.  Now we never explicitly touched `x`: it used to be `[500, 501, 502]`, but when we print it out it's `[500, 600, 502]`.

This is a feature of Python - you can have two variables pointing at the same container, it is done all the time and it can be quite useful, but it can lead to this sort of "action at a distance" where changes to one variable affect the value of another.  But once you understand what is happening in memory, it isn't that surprising.

Finally, what happens if we assign y to a new list?

In [ ]:
y = [700, 800]

This is the standard assignment pattern, so this will create the object on the right and bind it to the specified object on the left. So we get new integer objects for 700 and 800, and a new list which has its first and second indices bound to those objects.  And then `y` is bound to that new list, breaking the previous binding for the variable, so `x` and `y` now refer to different objects.

Function Argument Passing
-------------------------

This is getting a little ahead of the discussion, since we haven't yet talked about functions in detail, but the same ideas work for understanding how arguments are passed into Python functions.  Lets look at some sample code.

We have a function which we are going to call which multiplies two objects:


In [ ]:
def mult(x, y):
    res = x * y
    return res

and we're going to call it from some code like this:

    a = [700, 800]
    b = 2
    c = mult(a, b)

So when we run the first two lines

In [ ]:
a = [700, 800]
b = 2

we create some objects bound in our namespace, in this case a list object bound to `a` which in turn binds two integers, and an integer bound to `b`.

When the line

In [ ]:
c = mult(a, b)

is executed, the references to the objects bound to `a` and `b` are taken, and a new namepsace is created for the function, with the argument variables bound to the corresponding objects, so `x` gets bound to the same list object as `a` and `y` gets bound to the same integer object as `b`.

So one way to think about function arguments is that they are "passed by assignment".

In a similar way, if you create a class instance with code something like this:

In [ ]:
class Person(object):
    # nothing here in this simple example
    pass

a = [700, 800]
b = 2
person = Person()
person.age = b
person.values = a

where we have assigned to the attributes of the `Person` instance, then this works in much the same way, with the `person` object having its own namespace. The variable `age` in that namespace binds to the same integer object as `b` in our main namespace; and the variable `values` in that namespace binds to the same list as the variable `a`.

Playing with Executing Code
---------------------------

Let's go back to our original example, and play around with executing it manually in Python.  This is one of the "secret handshakes" of dynamic language programmers: playing with creating code strings and compiling and executing them yourself.  In this case this is a nice exercise because it starts to give us an idea of what these namespaces that we have been talking about actually are (see if you can guess what data structure is used before reading on).

Let's create a string containing the code that we just wrote:

In [ ]:
code = """
x = 500
y = x
y = 'foo'
print(x)
"""
code

This is just a snippet of text, but Python can execute text.

We need one other thing, which is our namespace, and that's just a dictionary.

In [ ]:
namespace = {}

In fact there will be two namespaces, this one, and the global namespace which we aren't so interested in, so we'll just give it a temporary value.  But we can then say:

In [ ]:
exec code in {}, namespace

You can see that when we executed the code, 500 was printed, as you would expect if you had typed the code in IPython.

But if we look at our namespace:

In [ ]:
namespace

we see that it has our variables and values in it.

So we've just done manually what every command interpreter, like IPython, or the Python command line interpreter, does under the covers.

This is a neat party trick, but it can come in handy when you want to provide end users with the ability to evaluate expressions in your code, or extend the functionality of your application on the fly.

So hopefully this has given you some insight into how Python code execution works, and in particular how assignment works, and the fact that it is very much just assignment into a dictionary, and this knowledge will hopefully become useful when you are facing a late-night debugging session.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com